In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col, lower

In [2]:
spark = SparkSession.builder.master('yarn') \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.3.1")\
    .config('spark.sql.caseSensitive',True) \
    .appName('dataTreatment') \
    .getOrCreate()
spark

/home/hadoop/notebook/jupyter/lib64/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


----

In [3]:
def toOracle(df, table:str):
    df.write \
        .format("jdbc") \
        .option("url", f"jdbc:postgresql://{url}:{port}/{db}") \
        .option("dbtable", table) \
        .option("driver", "org.postgresql.Driver") \
        .option("user", user) \
        .option("password", password) \
        .mode("append") \
        .save()

In [4]:
labels = [
    ('ano', IntegerType()),
    ('trim', IntegerType()),
    ('mes', IntegerType()),
    ('dataatendimento', StringType()),
    ('cod_regiao', IntegerType()),
    ('regiao', StringType()),
    ('uf', StringType()),
    ('cod_tipoatendimento', IntegerType()),
    ('descricaotipoatendimento', StringType()),
    ('cod_assunto', IntegerType()),
    ('descricaoassunto', StringType()),
    ('grupoassunto', StringType()),
    ('cod_problema', IntegerType()),
    ('descricaoproblema', StringType()),
    ('grupoproblema', StringType()),
    ('sexo', StringType()),
    ('faixaetariaconsumidor', StringType()),
    ('cepconsumidor', IntegerType())
]
schema = StructType([StructField (x[0], x[1], True) for x in labels])

In [5]:
df = spark.read \
    .option("recursiveFileLookup",True) \
    .option("header",True) \
    .option("sep",",") \
    .csv("/user/hadoop/datalake/reclamacoes/", schema=schema) \
    .persist(StorageLevel.MEMORY_ONLY)
print("Número de linhas no nosso DataFrame -> {} .".format(df.count()))
print(f"Schema do nosso {type(df)} :",end='\n \n')
df.printSchema()

Número de linhas no nosso DataFrame -> 6089960 .
Schema do nosso <class 'pyspark.sql.dataframe.DataFrame'> :
 
root
 |-- ano: integer (nullable = true)
 |-- trim: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dataatendimento: string (nullable = true)
 |-- cod_regiao: integer (nullable = true)
 |-- regiao: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- cod_tipoatendimento: integer (nullable = true)
 |-- descricaotipoatendimento: string (nullable = true)
 |-- cod_assunto: integer (nullable = true)
 |-- descricaoassunto: string (nullable = true)
 |-- grupoassunto: string (nullable = true)
 |-- cod_problema: integer (nullable = true)
 |-- descricaoproblema: string (nullable = true)
 |-- grupoproblema: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- faixaetariaconsumidor: string (nullable = true)
 |-- cepconsumidor: integer (nullable = true)




### - Imprimimos uma amostra do nosso df

In [7]:
df.sample(0.5).show(3, truncate=False)

+----+----+---+-----------------------+----------+-------+---+-------------------+---------------------------------------+-----------+-------------------------+-------------------------------+------------+------------------------------------------------------------------------------------+----------------------+----+---------------------+-------------+
|ano |trim|mes|dataatendimento        |cod_regiao|regiao |uf |cod_tipoatendimento|descricaotipoatendimento               |cod_assunto|descricaoassunto         |grupoassunto                   |cod_problema|descricaoproblema                                                                   |grupoproblema         |sexo|faixaetariaconsumidor|cepconsumidor|
+----+----+---+-----------------------+----------+-------+---+-------------------+---------------------------------------+-----------+-------------------------+-------------------------------+------------+------------------------------------------------------------------------------------+

### - Perceba que dependendo da quantidade de colunas há a quebra da tabela para a próxima linha, uma soluçao seria ->

In [8]:
df.sample(0.5).show(1, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------------
 ano                      | 2019                                    
 trim                     | 1                                       
 mes                      | 1                                       
 dataatendimento          | 2019-01-25 15:41:50.000                 
 cod_regiao               | 3                                       
 regiao                   | Sudeste                                 
 uf                       | ES                                      
 cod_tipoatendimento      | 6                                       
 descricaotipoatendimento | Atendimento Preliminar                  
 cod_assunto              | 187                                     
 descricaoassunto         | Telefonia Celular                       
 grupoassunto             | Telefonia Celular                       
 cod_problema             | 3680                                    
 descricaoproblema        | Recusa

### - Uma alternativa de exibição é possível no Jupyter usando Spark>=2.4.0 :

In [9]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # Ativamos a FLAG
spark.conf.set("spark.sql.repl.eagerEval.truncate", '0') # Setamos o limite de caracteres antes de truncar '0' deixa ilimitado
#spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 3) 
df.limit(3)

ano,trim,mes,dataatendimento,cod_regiao,regiao,uf,cod_tipoatendimento,descricaotipoatendimento,cod_assunto,descricaoassunto,grupoassunto,cod_problema,descricaoproblema,grupoproblema,sexo,faixaetariaconsumidor,cepconsumidor
2019,1,1,2019-01-25 15:58:33.000,3,Sudeste,ES,2,Simples Consulta,63,Financeira,Financeira,1099,"Contrato (não cumprimento, alteração, transferencia, irregularidade, rescisão, etc.)",Problemas com Contrato,F,mais de 70 anos,29172500
2019,1,1,2019-01-25 15:57:31.000,3,Sudeste,ES,3,CIP (Carta de Informações Preliminares),76,Outros Contratos,Contratos Financeiros Diversos*,1112,"Contrato (não cumprimento, alteração, transferencia, irregularidade, rescisão, etc.)",Problemas com Contrato,M,entre 21 a 30 anos,29158038
2019,1,1,2019-01-25 15:42:34.000,3,Sudeste,ES,2,Simples Consulta,58,Consórcio de Motocicletas,Consórcios,1094,"Contrato (não cumprimento, alteração, transferencia, irregularidade, rescisão, etc.)",Problemas com Contrato,M,entre 31 a 40 anos,29135000


In [13]:
# Listamos ano, trimestre e mês das reclamaçoes. Perceba as 'n' maneiras de referenciar uma coluna ou range de colunas(slice)
df.select('ano',df['trim'], df[2]).distinct().orderBy(df.columns[0:3], ascending=True).show(3, vertical=False)

+----+----+---+
| ano|trim|mes|
+----+----+---+
|2019|   1|  1|
|2019|   1|  2|
|2019|   1|  3|
+----+----+---+
only showing top 3 rows



In [26]:
df.filter((lower(col('descricaoassunto')).like("%carr%")) | (lower(col('descricaoassunto')).like("%auto%"))).select("descricaoassunto").distinct()

descricaoassunto
Carro Nacional Zero ( Montadora )
Carro Usado
"Consórcio de Automóveis ou Automotores, Utilitários, Caminhonetes"
Carro Importado
"Automóvel - Locação / Assistência Automobilística ( Clube do Automóvel, Etc. ) / Auto Escola"
"Oficinas ( Mecânica, Funilaria, Auto-Elétrica, Borracharia )"
Seguro de Automóvel
"Combustível Automotivo ( Gasolina, Álcool, Diesel, Gás )"
